In [1]:
# lets start by getting pandas, regex, and numpy
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import helper as helper

# 2. Background # 
This is the stuff we need to make this work. </br>
So we need the relevant dataframes, and the relevant functions. </br>
The functions are: 
1) 'ingredient_divider' which takes a string and returns the percengtage of ingredietns which are covered
2) 'matching_ingredient' which takes a url, and updates the recipes dataframe with the ingredients and the percentage of ingredients covered
3) 'matching_ingredient_df' which takes a df, and updates the recipes dataframe with the ingredients and the percentage of ingredients covered
4) 'check_fridge_availability' which takes an ingredient name and amount needed, and returns the amount available in the fridge
5) 'calculate_cost_and_remaining' which takes a row and returns the cost and remaining amount of ingredients needed
6) 'calculate_cost_of_urls' which takes a list of urls and returns the total cost of the ingredients needed

In [2]:
recipes = pd.read_csv('recipes.csv')
recipes

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,Indisk vegetarbowl med rispilaf og sprød blomk...,0.0,1,0,1,7714.0,0.173913
1,https://www.valdemarsro.dk/pasta-med-graeskar/,Pasta med græskar og masser af god smag - opsk...,0.0,1,0,1,6415.0,0.181818
2,https://www.valdemarsro.dk/blomkaal-og-halloum...,Blomkål og halloumi i sursød sauce med dejlig ...,0.0,1,0,1,6278.0,0.125000
3,https://www.valdemarsro.dk/graeskarmuffins/,Græskarmuffins - opskrift på saftige muffins m...,0.0,0,0,0,5100.0,0.133333
4,https://www.valdemarsro.dk/vegetar-taco/,Vegetar taco med krydret bagt blomkål og mango...,0.0,1,0,1,4967.0,0.062500
...,...,...,...,...,...,...,...,...
2474,https://www.valdemarsro.dk/november-glaeder-je...,I november glæder jeg mig til ... - Valdemarsr...,NaN,0,0,0,0.0,0.000000
2475,https://www.valdemarsro.dk/tag/peber/,peber Archives - Valdemarsro\nOpskrifter\nOpsk...,NaN,0,0,0,0.0,0.000000
2476,https://www.valdemarsro.dk/forretter/,Forretter – Opskrifter på nemme og lækre forre...,NaN,0,0,0,0.0,0.000000
2477,https://www.valdemarsro.dk/hjemmelavet-kebab-k...,Hjemmelavet kebabkrydderi som giver en skøn sm...,0.8,0,0,0,0.0,0.000000


In [3]:
recipes = pd.read_csv('recipes.csv')
ingredients_df = pd.read_csv('ingredients_df.csv')
pantry = pd.read_csv('pantry.csv')
# lets go to the download folder and import the ingredients_3.csv file
conversion = pd.read_csv('conversion.csv')
fridge = pd.read_csv('fridge.csv')

In [4]:
# now lets pick a random vegeterian recipe and see how many matches it has
url = recipes[recipes.vegeterian == True].sample().urls.values[0]
# now lets make a column in the recipe dataframe that shows the matching ingredients
# we will do this by giving a function a url, and then it will iterate through the pantry, matching the ingredients against the original recipe
# then it will count how many ingredients match, and return that number

# now lets pick a random vegeterian recipe and see how many matches it has
url = recipes[recipes.vegeterian == True].sample().urls.values[0]

url = 'https://www.valdemarsro.dk/fransk-bondesalat/'
# now lets make a column in the recipe dataframe that shows the matching ingredients
recipes = helper.matching_ingredients(url, pantry = pantry, recipes = recipes)
# now lets see how many matches it has
recipes[recipes.dinner == True].sort_values(by='matches', ascending=False).head(10)

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
483,https://www.valdemarsro.dk/fransk-bondesalat/,Fransk bondesalat - opskrift på en virkelig fa...,0.0,1,0,1,0.0,1.000000
589,https://www.valdemarsro.dk/salade-nicoise/,Salade Nicoise - Opskrift på den klassiske sal...,0.0,1,0,0,40.0,0.727273
471,https://www.valdemarsro.dk/salade-lyonnaise/,Salade Lyonnaise Opskrift - en skøn frokost sa...,0.0,1,0,0,70.0,0.692308
852,https://www.valdemarsro.dk/blt-sandwich/,BLT sandwich – opskrift på sandwich med bacon ...,0.0,1,0,0,59.0,0.625000
706,https://www.valdemarsro.dk/salade-chevre-chaud/,Salade Chevre Chaud - Opskrift på lækker salat...,0.0,1,0,1,50.0,0.600000
1040,https://www.valdemarsro.dk/aeggekage-opskrift/,Æggekage med kartofler og purløg - opskrift\nO...,NaN,1,0,0,39.0,0.571429
368,https://www.valdemarsro.dk/bagt-torsk-med-kape...,Bagt torsk med kapersvinaigrette og grønt\nOps...,0.0,1,0,0,157.0,0.545455
1113,https://www.valdemarsro.dk/svinekrone/,Svinekrone med honning og timian – opskrift ti...,0.0,1,0,0,0.0,0.500000
773,https://www.valdemarsro.dk/pariserbof/,Pariserbøf - Opskrift på den bedste klassiske ...,0.0,1,0,0,97.0,0.500000
720,https://www.valdemarsro.dk/roedbedeboeffer-ala...,Rødbedebøffer - Opskrift på lækre vegetar bøff...,0.0,1,0,1,68.0,0.461538


In [8]:
fridge = pd.read_clipboard()
fridge.columns = ['ingredients', 'amounts']
fridge = fridge[fridge.amounts > 0]

fridge

,ingredients,amounts
0,løg,3.0
1,hvidløg,8.0
2,rødvin,740.0
3,olivenolie,970.0
8,grøntsagsbouillon,150.0
9,soja,200.0
11,mango,125.0
12,rødløg,6.5
13,jalapenos,45.0
15,lime,4.0


In [129]:
test[['ingredients', 'remaining', 'units']].to_clipboard()

In [118]:
urls

['Kartoffelsalat med honning-sennepsdressing og grønne asparges',
 'https://www.valdemarsro.dk/fransk-bondesalat/']

In [121]:
test = calculate_cost_of_urls([urls[1]])


skiver bacon


,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,Indisk vegetarbowl med rispilaf og sprød blomk...,0.0,1,0,1,7714.0,0.173913
1,https://www.valdemarsro.dk/pasta-med-graeskar/,Pasta med græskar og masser af god smag - opsk...,0.0,1,0,1,6415.0,0.181818
2,https://www.valdemarsro.dk/blomkaal-og-halloum...,Blomkål og halloumi i sursød sauce med dejlig ...,0.0,1,0,1,6278.0,0.125000
3,https://www.valdemarsro.dk/graeskarmuffins/,Græskarmuffins - opskrift på saftige muffins m...,0.0,0,0,0,5100.0,0.133333
4,https://www.valdemarsro.dk/vegetar-taco/,Vegetar taco med krydret bagt blomkål og mango...,0.0,1,0,1,4967.0,0.062500
...,...,...,...,...,...,...,...,...
3255,Kikærte og mango curry med basmati ris,Navn: Kikærte og mango curry med basmati ris\n...,0.0,0,0,0,0.0,0.192308
3256,Kikærte- og kyllingefad med avocado og cherryt...,Navn: Kikærte- og kyllingefad med avocado og c...,0.0,0,0,0,0.0,0.125000
3257,Persille-ris med kikærter og yoghurt-dressing,Navn: Persille-ris med kikærter og yoghurt-dre...,0.0,0,0,0,0.0,0.062500
3258,Avocado og kikærte tacos,Navn: Avocado og kikærte tacos\n\nIngredienser...,0.0,0,0,0,0.0,0.076923


In [37]:
# now we need to update the values in the pantry dataframe
# so we need to iterate through the ingredients_df and for each ingredient, we need to find the matching ingredient in the pantry dataframe
# and then overwrite the values in the pantry dataframe with the values in the ingredients_df dataframe

for ingredient in ingredients_df.ingredients.unique():
    # print(ingredient)
    # print(ingredients_df[ingredients_df.ingredients == ingredient].local_price.values[0])
    pantry.loc[pantry.ingredients == ingredient, 'local_price'] = ingredients_df[ingredients_df.ingredients == ingredient].price.values[0]
    pantry.loc[pantry.ingredients == ingredient, 'purchase_size'] = ingredients_df[ingredients_df.ingredients == ingredient].purchase_size.values[0]
    pantry.loc[pantry.ingredients == ingredient, 'purchase_unit'] = ingredients_df[ingredients_df.ingredients == ingredient].purchase_unit.values[0]

In [40]:
pantry.sort_values(by = 'local_price', ascending=False).head(10)

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
4768,https://www.valdemarsro.dk/gloegg/,0.75,dl,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.000000
5937,https://www.valdemarsro.dk/summer-breeze-cockt...,2.00,cl,gin,ml,0.0,200.0,750.0,200.0,False,False,0.000000
6347,https://www.valdemarsro.dk/raasyltede-blommer/,1.00,dl,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.000000
16579,http://www.valdemarsro.dk/romkugler/,1.00,spsk,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.000000
3681,https://www.valdemarsro.dk/fedtebroed/,1.50,spsk,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.142857
5123,https://www.valdemarsro.dk/cocktail-rom-appels...,1.50,cl,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.000000
4431,https://www.valdemarsro.dk/gluhwein/,0.25,dl,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.000000
3434,https://www.valdemarsro.dk/traestammer/,2.00,spsk,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.000000
16809,https://www.valdemarsro.dk/dark-n-stormy/,2.50,cl,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.000000
959,https://www.valdemarsro.dk/valnoeddelagkage/,1.50,spsk,mørk rom,ml,0.0,200.0,750.0,200.0,False,False,0.153846


In [7]:

pantry['local_price'] = pantry.apply(set_individual_price, axis=1)

TypeError: set_individual_price() missing 1 required positional argument: 'conversion'

In [56]:

local_urls = recipes[(recipes.dinner == True) & (recipes.vegeterian == True)].sort_values('matches', ascending=False).head(10).sample(1).urls.tolist()


local_urls = find_recipes(local_urls=local_urls, price_cap=500, dishes=2)

history = pd.DataFrame(columns=['dish_1', 'dish_2', 'dish_3', 'price'])

old_urls = []


for i in range(3):
    local_urls = find_recipes(local_urls=local_urls, price_cap=500, dishes=3)
    if len(local_urls) == 1:
        local_urls = recipes[(recipes.dinner == True) & (recipes.vegeterian == True) & (~recipes.urls.isin(old_urls))].sort_values('matches', ascending=False).head(10).sample(1).urls.tolist()
        find_recipes(local_urls=local_urls, price_cap=500, dishes=3)
    local_groceries = calculate_cost_of_urls(local_urls)
    price = local_groceries['cumsum'].max()
    # fridge = set_fridge(local_groceries)
    sample = local_urls
    history.loc[len(history)] = [local_urls[0], local_urls[1], local_urls[2], price]
    old_urls = local_urls
    matching_ingredient_df(fridge)
    local_urls = recipes[(recipes.dinner == True) & (recipes.vegeterian == True) & (~recipes.urls.isin(old_urls))].sort_values('matches', ascending=False).head(10).sample(1).urls.tolist()

sample = calculate_cost_of_urls(history.loc[0].values.tolist()[:3])

i tried a thousand times
i tried a thousand times
i tried a thousand times
i tried a thousand times


IndexError: list index out of range

In [18]:
ingredients_df.to_csv('ingredient_df.csv', index=False)
recipes.to_csv('recipes_2.csv', index=False)
pantry.to_csv('pantry.csv', index=False)
# fridge.to_csv('fridge.csv', index=False)
# history.to_csv('history.csv', index=False)
conversion.to_csv('conversion.csv', index=False)